In [61]:
from collections import defaultdict
import json
import heapq


In [62]:
recipes = []

bow = defaultdict(int)
ingredients_total = set()


counter = 0
with open('./data.json') as json_file:
    data = json.load(json_file)
    for p in data:
        recipes.append(p)
        for word in p['instructions'].lower().split(' '):
            bow[word] +=1

        for ing in p['ingredients']:
            if ing['role'] == 'flavor texture':
                ingredients_total.add(ing['name'])


bow = {k: v for k, v in sorted(bow.items(), key=lambda item: item[1], reverse=True)}



In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [recipe['title'] for recipe in recipes]
corpus_index = {k: v for k, v in enumerate(corpus)}
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


In [80]:
def merge_titles(index1, index2, ingredients_new_recipe):
    #print(vectorizer.get_feature_names())
    ingredients_new_recipe = [x['name'].lower() for x in ingredients_new_recipe]
    print(ingredients_new_recipe)

    feature_names = vectorizer.get_feature_names()
    feature_index = X[index1,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [X[index1, x] for x in feature_index])
    
    pq = []
    name1 = ''
    first_high_score = -1
    first_characteristics = []
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        first_characteristics.append(w)
        for ing in ingredients_new_recipe:
            if w in ing and first_high_score < s:
                name1 = w + ' '
                
            else:
                if w != 'cookies': 
                    pq.append((s, w))


    print("---")
    common_characteristics = []
    pq2 = []
    name2 = ' '
    second_high_score = -1
    feature_index = X[index2,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [X[index2, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:

        if w == 'cookies':
            continue

        if w in first_characteristics and w != 'cookies':
            common_characteristics.append(w)

        for ing in ingredients_new_recipe:
            if w in ing and second_high_score < s:
                name2 = w + ' '
            else: 
                if w != 'cookies': 
                    pq2.append((s, w))

    sorted(pq, reverse=True)
    sorted(pq2, reverse=True)

    words = set([*common_characteristics, pq[0][1], pq[1][1]])

    if len(words) > 0:
        words =  ' '.join(words) + ' '
    else:
        words = ''

    ans = words + name1 + name2 + 'cookies'
    return ans


print(recipes[3]['title'].lower())
print(recipes[5]['title'].lower())
merge_titles(3, 5, recipes[3]['ingredients'])

award winning soft chocolate chip cookies
big soft ginger cookies
['butter', 'butter flavored shortening', 'packed light brown sugar', 'white sugar', 'egg', 'vanilla extract', 'all-purpose flour', 'baking soda', 'ground cinnamon', 'ground clove', 'salt', 'rolled oat', 'raisin']
---


'chip soft award chocolate winning  cookies'

In [91]:
a = ['ok']
def separate_enumeration(a):
    if len(a) > 1:
        instruction = 'Add ' + ', '.join(a[:-1]) + ' and '  + a[-1] + '. '
    else:
        instruction = 'Add ' + a[0] + '. '
    return instruction

print(separate_enumeration(a))

Add ok. 
